In [5]:
!git clone https://github.com/Js-Hwang1/LST.git

Cloning into 'LST'...
remote: Enumerating objects: 859, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 859 (delta 18), reused 32 (delta 12), pack-reused 816 (from 1)
Receiving objects: 100% (859/859), 387.13 MiB | 42.25 MiB/s, done.
Resolving deltas: 100% (408/408), done.
Updating files: 100% (122/122), done.


In [6]:
!nvidia-smi

Sat Dec 27 06:30:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
!ls

sample_data
